In [1]:
!python -m pip install -U "torch>=2.2,<3.0" torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!python -m pip install -U sentence-transformers scikit-learn pandas numpy joblib

Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 136.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 67.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 69.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 186.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2

[notice] A new release of pip is available: 25.0.1 -> 25.2
[not

In [2]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 84.5 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
# ============================================
# 설치 (Runpod A40 / 로컬)
# ============================================
# CPU 전용
# python -m pip install -U sentence-transformers "torch>=2.2,<3.0" scikit-learn pandas numpy joblib xgboost

# GPU (CUDA 12.1, Runpod A40)
# python -m pip install -U "torch>=2.2,<3.0" torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# python -m pip install -U sentence-transformers scikit-learn pandas numpy joblib xgboost
# ============================================

import os, time
import numpy as np
import pandas as pd
from collections import Counter

import torch
from sentence_transformers import SentenceTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report

import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

# -------------------------------
# 설정
# -------------------------------
DATA_CSV = "perfumes_huggingface.csv"  # 경로 맞게 수정
MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"  # 또는 distiluse-base-multilingual-cased-v2
TOP_K = 3
RARE_MIN_COUNT = 10

# -------------------------------
# 유틸
# -------------------------------
def split_labels(s: str):
    s = str(s)
    for sep in [",", "|", "/", ";"]:
        s = s.replace(sep, " ")
    return [t.strip() for t in s.split() if t.strip()]

def encode_with_auto_batch(embedder: SentenceTransformer, texts, init_bs=1024, min_bs=64):
    """CUDA OOM 시 배치 크기를 줄여가며 안전하게 임베딩"""
    bs = init_bs
    Xs = []
    i = 0
    n = len(texts)
    while i < n:
        j = min(i + bs, n)
        chunk = texts[i:j]
        try:
            emb = embedder.encode(chunk, batch_size=bs, convert_to_numpy=True, show_progress_bar=False)
            Xs.append(emb)
            i = j
        except RuntimeError as e:
            if "CUDA out of memory" in str(e) and bs > min_bs:
                torch.cuda.empty_cache()
                bs = max(min_bs, bs // 2)
                print(f"[WARN] CUDA OOM → batch_size 축소: {bs}")
                continue
            raise
    return np.vstack(Xs)

# -------------------------------
# 1) 데이터 로드 & 전처리
# -------------------------------
df = pd.read_csv(DATA_CSV, sep="|", engine="python", on_bad_lines="skip")
df = df[~df["description"].isna()].copy()
df["labels"] = df["fragrances"].apply(split_labels)

# 희소 라벨 제거
cnt = Counter([l for L in df["labels"] for l in L])
rare = {k for k, v in cnt.items() if v <= RARE_MIN_COUNT}
df["labels"] = df["labels"].apply(lambda L: [l for l in L if l not in rare])
df = df[df["labels"].map(len) > 0].copy()

# 타깃 인코딩
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df["labels"])

# -------------------------------
# 2) 데이터 분할
# -------------------------------
X_train_text, X_val_text, y_train, y_val = train_test_split(
    df["description"].tolist(), Y, test_size=0.2, random_state=42
)

# -------------------------------
# 3) 임베딩
# -------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[Device] {device}")

embedder = SentenceTransformer(MODEL_NAME, device=device)
init_bs = 1024 if device == "cuda" else 128

t0 = time.perf_counter()
X_train = encode_with_auto_batch(embedder, X_train_text, init_bs=init_bs, min_bs=64)
t1 = time.perf_counter()
print(f"[Embed] train: {X_train.shape} | time: {t1 - t0:.2f}s")

t0 = time.perf_counter()
X_val = encode_with_auto_batch(embedder, X_val_text, init_bs=init_bs, min_bs=64)
t1 = time.perf_counter()
print(f"[Embed] valid: {X_val.shape} | time: {t1 - t0:.2f}s")

# -------------------------------
# 4) XGBoost OvR 학습
# -------------------------------
clf = OneVsRestClassifier(
    xgb.XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        use_label_encoder=False,
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=-1,
        tree_method="hist"
    )
)

t0 = time.perf_counter()
clf.fit(X_train, y_train)
t1 = time.perf_counter()
print(f"[Train] OvR-XGBoost: {t1 - t0:.2f}s")

y_val_proba = clf.predict_proba(X_val)

# -------------------------------
# 5) 라벨별 Threshold 최적화 (0.2~0.5 Grid Search)
# -------------------------------
thresholds = {}
y_val_pred_opt = np.zeros_like(y_val)

for i, label in enumerate(mlb.classes_):
    best_thr, best_f1 = 0.5, -1
    for thr in np.linspace(0.2, 0.5, 16):  # 0.2~0.5 구간
        pred = (y_val_proba[:, i] >= thr).astype(int)
        f1 = f1_score(y_val[:, i], pred, zero_division=0)
        if f1 > best_f1:
            best_thr, best_f1 = thr, f1
    thresholds[label] = best_thr
    y_val_pred_opt[:, i] = (y_val_proba[:, i] >= best_thr).astype(int)

print("\n[Best Thresholds per label]")
for k, v in thresholds.items():
    print(f"{k}: {v:.2f}")

# -------------------------------
# 6) 평가 (Threshold-based + Top-K-based)
# -------------------------------
print("\n=== Threshold-based ===")
print(f"Micro-F1: {f1_score(y_val, y_val_pred_opt, average='micro'):.4f}")
print(f"Macro-F1: {f1_score(y_val, y_val_pred_opt, average='macro'):.4f}")
print(f"Sample-F1: {f1_score(y_val, y_val_pred_opt, average='samples'):.4f}")
print("\n[classification_report @thr]")
print(classification_report(y_val, y_val_pred_opt, target_names=mlb.classes_, zero_division=0))

# Top-K
top_idx = np.argsort(-y_val_proba, axis=1)[:, :TOP_K]
y_val_topk = np.zeros_like(y_val_proba, dtype=int)
for i, idxs in enumerate(top_idx):
    y_val_topk[i, idxs] = 1

print("\n=== Top-K-based ===")
print(f"Micro-F1: {f1_score(y_val, y_val_topk, average='micro'):.4f}")
print(f"Macro-F1: {f1_score(y_val, y_val_topk, average='macro'):.4f}")
print(f"Sample-F1: {f1_score(y_val, y_val_topk, average='samples'):.4f}")
print("\n[classification_report @topK]")
print(classification_report(y_val, y_val_topk, target_names=mlb.classes_, zero_division=0))

# -------------------------------
# 7) 예측 함수
# -------------------------------
def predict_multilingual(text: str, topk=3, thresholds=None):
    v = encode_with_auto_batch(embedder, [text], init_bs=64 if device=="cpu" else 256, min_bs=32)
    proba = clf.predict_proba(v)[0]

    if thresholds is not None:  # 라벨별 threshold 적용
        pick = [i for i, p in enumerate(proba) if p >= thresholds.get(mlb.classes_[i], 0.5)]
        if not pick:  # 아무 라벨도 안 나오면 Top-K fallback
            pick = np.argsort(-proba)[:topk]
    else:  # Top-K
        pick = np.argsort(-proba)[:topk]

    return [mlb.classes_[i] for i in pick]

# -------------------------------
# 예시 실행
# -------------------------------
print("\n[Example Prediction]")
print(predict_multilingual("바닷가에서 느껴지는 시원하고 약간 달콤한 향이 좋아요", topk=3, thresholds=thresholds))


[Device] cuda


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[Embed] train: (21001, 384) | time: 12.36s
[Embed] valid: (5251, 384) | time: 3.05s
[Train] OvR-XGBoost: 130.72s

[Best Thresholds per label]
Amber: 0.24
Aromatic: 0.20
Blossom: 0.20
Bouquet: 0.20
Carnation: 0.20
Citrus: 0.20
Classical: 0.22
Crisp: 0.20
Dry: 0.20
Floral: 0.32
Flower: 0.20
Fougère: 0.20
Fresh: 0.20
Fresher: 0.30
Fruity: 0.20
Gardenia: 0.20
Gourmand: 0.20
Green: 0.20
Iris: 0.46
Jasmine: 0.24
Lily: 0.26
Magnolia: 0.20
Mimosa: 0.20
Mossy: 0.20
Musk: 0.20
Orange: 0.48
Oriental: 0.20
Rich: 0.20
Richer: 0.32
Rose: 0.22
Soft: 0.20
Spicy: 0.20
Tuberose: 0.20
Valley: 0.46
Violet: 0.32
Water: 0.20
White: 0.20
Woods: 0.20
Woody: 0.20
of: 0.46
the: 0.46

=== Threshold-based ===
Micro-F1: 0.4935
Macro-F1: 0.2511
Sample-F1: 0.4896

[classification_report @thr]
              precision    recall  f1-score   support

       Amber       0.42      0.79      0.55      1738
    Aromatic       0.41      0.29      0.34       450
     Blossom       0.40      0.08      0.13        25
     Bouqu

In [ ]:
print("\n[Example Prediction]")
print(predict_multilingual("깨끗하게 빨래하고 말린 상쾌한 향", topk=3, thresholds=thresholds))


[Example Prediction]
['Fresher', 'Amber', 'Floral']


In [ ]:
print("\n[Example Prediction]")
print(predict_multilingual("바다향", topk=3, thresholds=thresholds))


[Example Prediction]
['Fresher', 'Woods', 'Classical']
